In [2]:
import numpy as np
import astropy.units as u
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord

## Part 1

In [100]:
data = Table.read('/d/scratch/ASTR5160/data/legacysurvey/dr9/south/sweep/9.0/sweep-180p020-190p025.fits')

In [103]:
target = SkyCoord(188.53667*u.deg, 21.04572*u.deg)
sample_coord = SkyCoord(data['RA'], data['DEC'])

In [104]:
index = np.argmin(sample_coord.separation(target))
obj = data[index]

In [105]:
print(obj['TYPE']+', guess it means galaxy')

EXP, guess it means galaxy


## Part 2

In [106]:
flag_all_g = 2 ** 5
flag_all_r = 2 ** 6
flag_all_z = 2 ** 7

res = 3 - (int(obj['MASKBITS'] & flag_all_g == 0) \
    + int(obj['MASKBITS'] & flag_all_r == 0) + int(obj['MASKBITS'] & flag_all_z == 0))

In [107]:
print(f'saturation happens at {res} bands')

saturation happens at 3 bands


## Well, the SDSS viewer says it is a star. It is definitely saturated!

## Part 3

In [3]:
files = ['/d/scratch/ASTR5160/data/legacysurvey/dr9/south/sweep/9.0/sweep-170p030-180p035.fits',
         '/d/scratch/ASTR5160/data/legacysurvey/dr9/south/sweep/9.0/sweep-180p025-190p030.fits',
         '/d/scratch/ASTR5160/data/legacysurvey/dr9/south/sweep/9.0/sweep-180p030-190p035.fits']

hdul = fits.open('/d/scratch/ASTR5160/data/legacysurvey/dr9/south/sweep/9.0/sweep-170p025-180p030.fits')
data = hdul[1].data
hdul.close()

all_data = np.vstack((data['RA'], data['DEC'], data['TYPE'], data['FLUX_R']/data['MW_TRANSMISSION_R']))

for i in files:
    hdul = fits.open(i)
    data = hdul[1].data
    hdul.close()

    tem = np.vstack((data['RA'], data['DEC'], data['TYPE'], data['FLUX_R']/data['MW_TRANSMISSION_R']))

    all_data = np.hstack((all_data, tem))

In [4]:
target = SkyCoord([180]*u.deg, [30]*u.deg)
sweep_sample = SkyCoord(all_data[0, :].astype(float)*u.deg, all_data[1, :].astype(float)*u.deg)
index_sweep, _, _, _ = target.search_around_sky(sweep_sample, 3*u.deg)

In [5]:
sweep_sample_selected = all_data[:, index_sweep]
detected = sweep_sample_selected[3].astype(float) > 0
sweep_sample_selected_detected = sweep_sample_selected[:, detected]

In [6]:
qso_mag = (lambda x: 22.5 - 2.5*np.log10(x))(sweep_sample_selected_detected[3].astype(float))

In [7]:
masked = (qso_mag < 20) & (sweep_sample_selected_detected[2] == 'PSF')
res = sweep_sample_selected_detected[:, masked]

In [8]:
print(f'I find {len(res[0])} objects after r band cut and PSF selected')

I find 40427 objects after r band cut and PSF selected


In [9]:
qso = np.loadtxt('/d/users/kaiwen/Downloads/qso.txt')
qso_sample = SkyCoord(qso[:, 0]*u.deg, qso[:, 1]*u.deg)

current_data = res[:2].transpose().astype(float)
current_sample = SkyCoord(current_data[:, 0]*u.deg, current_data[:, 1]*u.deg)

In [16]:
index_match, _, _, _ = current_sample.search_around_sky(qso_sample, 0.5*u.arcsec)

In [22]:
print(f'I find {len(current_data[index_match])} QSOs')

I find 280 QSOs


In [23]:
qso_selected = qso[index_match]
np.savetxt('/d/users/kaiwen/Downloads/qso_selected.txt', qso_selected)